In [22]:
# -*- coding: utf-8 -*-
import cx_Oracle as co
import pandas as pd
from pandas import DataFrame
import pandas.io.sql as sql              # Pandas를 이용한 쿼리 방식
import urllib.request
import json
import logging
from pandas.io.json import json_normalize
import logging.handlers

print("작업시작")


# 로거 인스턴스를 만든다
logger = logging.getLogger('mylogger')

# 포매터를 만든다
fomatter = logging.Formatter('[%(levelname)s|%(name)s:%(lineno)s] %(asctime)s > %(message)s')

# 스트림과 파일로 로그를 출력하는 핸들러를 각각 만든다.
fileHandler = logging.FileHandler('./myLoggerTest.log')
streamHandler = logging.StreamHandler()

# 각 핸들러에 포매터를 지정한다.
fileHandler.setFormatter(fomatter)
streamHandler.setFormatter(fomatter)

# 로거 인스턴스에 스트림 핸들러와 파일핸들러를 붙인다.
logger.addHandler(fileHandler)
logger.addHandler(streamHandler)

# 로거 인스턴스로 로그를 찍는다.
logger.setLevel(logging.DEBUG)
#logging.basicConfig(filename='./loggingtest.log',level=logging.DEBUG)

logger.info("=========================================")
logger.info("작업시작")
logger.info("=========================================")

try:
    # 오라클 Connect
    con = co.connect('ID/PWD@DATABASEIP/DATABASENAME or PORT')

    # 네이버에 입력할 키워드 조회 (in database)
    query_result = sql.read_sql("select COMPANY_NAME as brand_keyword from KEYWORD_TABLE' ",con)  

    # 쿼리 결과 건수 조회, 필요함????????????????????????????????????????
    row_cnt = query_result.count()

    #DataFrame 초기화하여 생성
    df=pd.DataFrame()    

    #네이버 ID 와 Secret code 정보
    client_id = "NAVER_SECRET_ID"
    client_secret = "NAVER_SECRET_KEY"


    # Oracle Franchise Info 테이블에 저장된 brand name 기반으로 Naver API 호출하여 결과값 가져옴
    for index, row in query_result.iterrows():
        cnt = 1
        brand_name = row['BRAND_KEYWORD']
        # print(brand_name)
        encText = urllib.parse.quote(brand_name)
        #JSON 형태의 결과값을 리턴 받는다. Naver API는 JSON과 XML 형태의 리턴값을 준다.
        url = 'https://openapi.naver.com/v1/search/local.json?display=100&query=' + encText   + '&start=%d' % cnt   #display 100으로 바꿀것
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        # print(rescode)
        response_body = response.read()
        response_dcd = response_body.decode('utf-8')
        # JSON 형태의 파일을 Dataframe 형태로 전환
        data = json.loads(response_dcd)
    

        #Naver API에 해당 검색어 조회시 몇개의 결과값이 리턴되는지 총 리턴갯수 저장
        tot_cnt = data['total']
        # print('tot_cnt:' + str(tot_cnt))
        # df_name = 'df_' + str(df_cnt)
        # df=pd.DataFrame(data['items'])        #아래 json_normalize와 같은 역할을 한다.
        # 전체 JSON 결과값을 받은 DataFrame에서 Item에 저장되어 있는 값만 다시 저장
        df_name=json_normalize(data['items'])
        # 검색한 키워드를 DataFrame 내에 같이 저장한다.
        df_name['keyword'] = brand_name
        # [i.replace(u"\u2022", '').replace('<b>','').replace('</b>','').replace(u'\xa0','').replace('\xf9','').replace('&amp;','') for i in df_name]
        # df_name['description'] = df_name['description'].apply(lambda x: x.decode('unicode_escape').encode('euc-kr', 'ignore').strip())
        # print(df_name)

                # print(aaa)
            # print(str.replace(row['title'],'<b>',''))
        df = df.append(df_name)
        # print(df)
        #####################################################################################################
        # Naver API는 Max 1000개의 결과값만 주고 나머지는 Bad Request로 처리한다.
        #####################################################################################################
        if tot_cnt>1000:
            tot_cnt=1000
        for icnt in range(101,tot_cnt,100):
            #JSON 형태의 결과값을 리턴 받는다. Naver API는 JSON과 XML 형태의 리턴값을 준다.
            url = 'https://openapi.naver.com/v1/search/local.json?display=100&query=' + encText   + '&start=%d' % icnt   #display 100으로 바꿀것
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            # print(rescode)
            response_body = response.read()
            response_dcd = response_body.decode('utf-8')

            # JSON 형태의 파일을 Dataframe 형태로 전환
            data = json.loads(response_dcd)
            # data
            # df=pd.DataFrame(data['items'])
            # print(df)
            # aaa= pd.read_json(response_dcd)
            # xxx=dict(aaa['items'])
            # xxx

            # #Naver API에 해당 검색어 조회시 몇개의 결과값이 리턴되는지 총 리턴갯수 저장
            # tot_cnt = data['total']
            # print(tot_cnt)
            # df_name = 'df_' + str(df_cnt)
            # df=pd.DataFrame(data['items'])        #아래 json_normalize와 같은 역할을 한다.
            # 전체 JSON 결과값을 받은 DataFrame에서 Item에 저장되어 있는 값만 다시 저장
            df_name=json_normalize(data['items'])
            # 검색한 키워드를 DataFrame 내에 같이 저장한다.
            df_name['keyword'] = brand_name
            # [i.replace(u"\u2022", '').replace('<b>','').replace('</b>','').replace(u'\xa0','').replace('\xf9','').replace('&amp;','') for i in df_name]
            # print(df_name)

            df = df.append(df_name)
            # print(df)
            # print(df_name)
    cols = df.columns.tolist()
    # cols
    cols.insert(0, cols.pop(cols.index('keyword')))
    cols.insert(1, cols.pop(cols.index('title')))
    cols.insert(2, cols.pop(cols.index('category')))
    cols.insert(3, cols.pop(cols.index('description')))
    cols.insert(4, cols.pop(cols.index('telephone')))
    cols.insert(5, cols.pop(cols.index('roadAddress')))
    cols.insert(6, cols.pop(cols.index('address')))
    cols.insert(7, cols.pop(cols.index('mapx')))
    cols.insert(8, cols.pop(cols.index('mapy')))
    cols.insert(9, cols.pop(cols.index('link')))
    # cols
    df = df.reindex(columns= cols)
    df.drop('description',axis=1,inplace=True)
    df.drop('link',axis=1,inplace=True)

    # df      
    # df.to_excel('test.xlsx',index=False)
    for index, row in df.iterrows():
        for column in df:
            row[column]=str.replace(row[column],'<b>','')
            row[column]=str.replace(row[column],'</b>','')
            row[column]=str.replace(row[column],u'[^\x00-\x7F]','')
            row[column]=str.replace(row[column],'&amp;','')
            row[column]=str.replace(row[column],u'\xa0','')
            row[column]=str.replace(row[column],u'\u2022','')
            row[column]=str.replace(row[column],u'\xf9','')
            row[column]=str.replace(row[column],u'\xa0','')
            row[column]=str.replace(row[column],u'\u30fb','')
            row[column]=str.replace(row[column],u'\xec','') 
            row[column]=str.replace(row[column],u'\u02c6','') 
            row[column]=str.replace(row[column],u'\u02dc','') 
            row[column]=str.replace(row[column],'&','')
            row[column]=str.replace(row[column],u'\xed','')
            row[column]=str.replace(row[column],u'\u201a','') 
            row[column]=str.replace(row[column],u'\xeb' ,'') 
            row[column]=str.replace(row[column],u'\u201e','') 
            row[column]=str.replace(row[column],u'\xa5','')  
            row[column]=str.replace(row[column],u'\xa3','') 
            row[column]=str.replace(row[column],u'\u2219','')
            row[column]=str.replace(row[column],u'\xe9','')
            # row[column]=str.encode(row[column],'euc-kr','replace').decode()




    cursor = con.cursor()
    exported_data = [tuple(x) for x in df.values]
    sql_query = "INSERT INTO STOREMST_TABLE (KEYWORD,TITLE,CATEGORY,TELEPHONE,ROADADDRESS,ADDRESS,MAPX,MAPY) VALUES (:1, :2, :3, :4, :5, :6, :7, :8); commit;"

    cursor.executemany(sql_query, exported_data)
    con.commit()

    cursor.close()
    # 오라클 close
    con.close()

except Exception as e:
    logger.error(e)
    logger.error(type(e))
    
    cursor.close()
    # 오라클 close
    con.close()

else:
    logger.info("작업완료")
    print("작업완료")
    

finally: 
    print("작업에러")
    logging.shutdown()

[INFO|<ipython-input-22-3f5c7112ccf2>:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|<ipython-input-22-3f5c7112ccf2>:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|<ipython-input-22-3f5c7112ccf2>:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|<ipython-input-22-3f5c7112ccf2>:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27,781 > =========================================
[INFO|mylogger:37] 2018-01-29 14:24:27

작업시작


[INFO|mylogger:38] 2018-01-29 14:24:27,865 > 작업시작
[INFO|mylogger:38] 2018-01-29 14:24:27,865 > 작업시작
[INFO|mylogger:38] 2018-01-29 14:24:27,865 > 작업시작
[INFO|mylogger:38] 2018-01-29 14:24:27,865 > 작업시작
[INFO|mylogger:38] 2018-01-29 14:24:27,865 > 작업시작
[INFO|<ipython-input-22-3f5c7112ccf2>:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|<ipython-input-22-3f5c7112ccf2>:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|<ipython-input-22-3f5c7112ccf2>:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|<ipython-input-22-3f5c7112ccf2>:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|mylogger:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|mylogger:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|mylogger:39] 2018-01-29 14:24:27,998 > =========================================
[INFO|mylogger:39] 2018-01-29 14:24:27,998 > ====

작업에러


NameError: name 'mylogger' is not defined